# Parameter Specification Workflow

This notebook explains how to specify parameters for PyLCM models. It covers:

1. Understanding `model.params_template` - what parameters your model needs
2. Specifying parameters at different levels (function, regime, model)
3. Parameter propagation - how higher-level specifications flow down
4. Error handling - what's not allowed and why

## Setup: A Simple Two-Regime Model

Let's create a simple consumption-savings model with two regimes: working and retired.
This will help us understand how parameters are organized.

In [ ]:
import jax.numpy as jnp

from lcm import AgeGrid, LinSpacedGrid, Model, Regime, categorical
from lcm.typing import ContinuousAction, ContinuousState, FloatND

In [ ]:
# Define regime identifiers
@categorical
class RegimeId:
    working: int
    retired: int

In [ ]:
# Utility functions
def utility_working(
    consumption: ContinuousAction, disutility_of_work: float
) -> FloatND:
    return jnp.log(consumption) - disutility_of_work


def utility_retired(consumption: ContinuousAction) -> FloatND:
    return jnp.log(consumption)

In [ ]:
# Auxiliary function: labor income (only in working regime)
def labor_income(wage: float) -> FloatND:
    return wage

In [ ]:
# State transitions (only in non-terminal regimes)
def next_wealth_working(
    wealth: ContinuousState,
    consumption: ContinuousAction,
    labor_income: FloatND,
    interest_rate: float,
) -> ContinuousState:
    return (1 + interest_rate) * (wealth - consumption) + labor_income

In [ ]:
# Regime transitions return the integer regime index
def next_regime_working() -> int:
    # Deterministically transition to retired (RegimeId.retired = 1)
    return RegimeId.retired

In [ ]:
# Constraint
def borrowing_constraint(
    consumption: ContinuousAction, wealth: ContinuousState
) -> FloatND:
    return consumption <= wealth

In [ ]:
# Define the grids
wealth_grid = LinSpacedGrid(start=1, stop=100, n_points=50)
consumption_grid = LinSpacedGrid(start=1, stop=100, n_points=50)

In [ ]:
# Define the regimes
working_regime = Regime(
    transition=next_regime_working,
    constraints={"borrowing_constraint": borrowing_constraint},
    functions={"utility": utility_working, "labor_income": labor_income},
    actions={"consumption": consumption_grid},
    states={
        "wealth": LinSpacedGrid(
            start=1, stop=100, n_points=50, transition=next_wealth_working
        )
    },
)

# Terminal regime: no transition needed (the model ends here)
retired_regime = Regime(
    transition=None,
    functions={"utility": utility_retired},
    constraints={"borrowing_constraint": borrowing_constraint},
    actions={"consumption": consumption_grid},
    states={"wealth": LinSpacedGrid(start=1, stop=100, n_points=50, transition=None)},
)

In [ ]:
# Create the model
model = Model(
    description="Simple two-regime consumption-savings model",
    ages=AgeGrid(start=60, stop=62, step="Y"),  # step is a string like "Y" (year)
    regimes={
        "working": working_regime,
        "retired": retired_regime,
    },
    regime_id_class=RegimeId,
)

## Understanding `model.params_template`

After creating a model, you can inspect `model.params_template` to see what parameters
are required. The template is organized hierarchically:

```
params_template
├── regime_name
│   ├── discount_factor: float  (always present)
│   ├── function_name
│   │   ├── param_1: type
│   │   └── param_2: type
│   └── another_function
│       └── param_3: type
└── another_regime
    └── ...
```

In [ ]:
import pprint

# View the params_template
pprint.pprint(dict(model.params_template))

Let's break this down:

**Working regime:**
- `discount_factor`: Required for discounting future utility
- `labor_income` function needs: `wage`
- `next_wealth_working` function needs: `interest_rate`
- `utility` function needs: `disutility_of_work`

**Retired regime (terminal):**
- `discount_factor`: Required for discounting
- `utility` function needs nothing (empty dict)

Notice that:
- `borrowing_constraint` appears in both regimes but needs no parameters
- `interest_rate` is only needed in the working regime (retired is terminal, no transitions)
- `wage` is only needed in the working regime

## Specifying Parameters: Three Levels

PyLCM allows you to specify parameters at three levels:

1. **Function level**: Most specific - directly in the function's dict
2. **Regime level**: Parameters propagate to all functions in that regime
3. **Model level**: Parameters propagate to all regimes and functions

This flexibility reduces repetition when the same parameter value is used across
multiple functions or regimes.

### Level 1: Function-Level Specification (Most Explicit)

This matches the structure of `params_template` exactly. Every parameter is specified
in its corresponding function dict.

In [ ]:
params_function_level = {
    "working": {
        "discount_factor": 0.95,
        "labor_income": {"wage": 10.0},
        "next_wealth_working": {"interest_rate": 0.04},
        "utility": {"disutility_of_work": 0.1},
    },
    "retired": {
        "discount_factor": 0.95,
    },
}

# This works!
V = model.solve(params_function_level, debug_mode=False)
print("Solved successfully with function-level params!")

### Level 2: Regime-Level Specification

Notice that `discount_factor` and `interest_rate` have the same value in both usages
within the working regime. We can specify them at the regime level instead:

In [ ]:
params_regime_level = {
    "working": {
        "discount_factor": 0.95,
        "interest_rate": 0.04,  # Propagates to next_wealth_working
        "wage": 10.0,  # Propagates to labor_income
        "disutility_of_work": 0.1,  # Propagates to utility
    },
    "retired": {
        "discount_factor": 0.95,
    },
}

V = model.solve(params_regime_level, debug_mode=False)
print("Solved successfully with regime-level params!")

### Level 3: Model-Level Specification

Now notice that `discount_factor` and `interest_rate` have the same values across
both regimes. We can specify them at the model level:

In [ ]:
params_model_level = {
    "discount_factor": 0.95,  # Propagates to both regimes
    "working": {
        "interest_rate": 0.04,  # Only needed in working regime
        "wage": 10.0,
        "disutility_of_work": 0.1,
    },
}

V = model.solve(params_model_level, debug_mode=False)
print("Solved successfully with model-level params!")

### Mixed-Level Specification

You can mix levels freely. For example, specify most parameters at the model level
but override specific ones at lower levels:

In [ ]:
params_mixed = {
    "discount_factor": 0.95,  # Model level: applies to both regimes
    "working": {
        "interest_rate": 0.05,  # Regime level
        "labor_income": {"wage": 12.0},  # Function level: override wage
        "disutility_of_work": 0.1,
    },
}

V = model.solve(params_mixed, debug_mode=False)
print("Solved successfully with mixed-level params!")

## What's NOT Allowed: Ambiguous Specifications

The key rule is: **you cannot specify the same parameter at multiple levels within
the same subtree**. This would be ambiguous - which value should be used?

PyLCM will raise an `InvalidNameError` if you try to do this.

### Error 1: Same parameter at model AND regime level

In [ ]:
from lcm.exceptions import InvalidNameError

params_ambiguous_model_regime = {
    "discount_factor": 0.95,  # Model level
    "working": {
        "discount_factor": 0.90,  # Also at regime level - AMBIGUOUS!
        "wage": 10.0,
        "interest_rate": 0.04,
        "disutility_of_work": 0.1,
    },
}

try:
    model.solve(params_ambiguous_model_regime)
except InvalidNameError as e:
    print(f"Error: {e}")

### Error 2: Same parameter at regime AND function level

In [ ]:
params_ambiguous_regime_function = {
    "discount_factor": 0.95,
    "working": {
        "interest_rate": 0.05,  # Regime level
        "next_wealth_working": {
            "interest_rate": 0.04,  # Also at function level - AMBIGUOUS!
        },
        "wage": 10.0,
        "disutility_of_work": 0.1,
    },
}

try:
    model.solve(params_ambiguous_regime_function)
except InvalidNameError as e:
    print(f"Error: {e}")

### Error 3: Same parameter at model AND function level

In [ ]:
params_ambiguous_model_function = {
    "wage": 15.0,  # Model level
    "discount_factor": 0.95,
    "working": {
        "labor_income": {
            "wage": 10.0,  # Also at function level - AMBIGUOUS!
        },
        "interest_rate": 0.04,
        "disutility_of_work": 0.1,
    },
}

try:
    model.solve(params_ambiguous_model_function)
except InvalidNameError as e:
    print(f"Error: {e}")

## Name Requirements

To enable unambiguous parameter propagation, PyLCM enforces naming rules:

1. **Regime names, function names, and argument names must not overlap with each
   other** (within the same category across regimes)
2. **Names cannot contain the separator `__`** (double underscore)

These rules are checked when the model is created. If violated, you'll get an error
during `Model()` initialization.

## Summary

| Level | Syntax | Propagation |
|-------|--------|-------------|
| Model | `{"param": value, ...}` | To all regimes and functions |
| Regime | `{"regime": {"param": value, ...}}` | To all functions in that regime |
| Function | `{"regime": {"func": {"param": value}}}` | Direct specification |

**Key Rules:**
- You can specify parameters at any level
- Higher-level specifications propagate down automatically
- The same parameter cannot be specified at multiple levels within a subtree
- Use `model.params_template` to see what parameters are needed